<a href="https://colab.research.google.com/github/123rc/natual_language_processing/blob/main/textclassifcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate
!pip install pandas==1.5.3
!pip install torch
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [3]:
import pandas as pd
import numpy as np
import torch
import pyarrow as pa
from datasets import Dataset

In [4]:
df=pd.read_csv('/content/gdrive/MyDrive/IMDB Dataset.csv')
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [5]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

In [6]:
def process_data(row):
  text=row['review']
  text=str(text)
  text=' '.join(text.split())
  encoding=tokenizer(text,padding='max_length',truncation=True,max_length=512)
  label=0
  if row['sentiment']=='positve':
    label+=1
  encoding['label']=label
  encoding['text']=text

  return encoding



In [7]:
processed_data=[]
for i in range(len(df[:1000])):
  processed_data.append(process_data(df.iloc[i]))
  

In [8]:
from sklearn.model_selection import train_test_split
new_df=pd.DataFrame(processed_data)
train,valid=train_test_split(new_df,test_size=0.2,random_state=42)


In [9]:
train=Dataset(pa.Table.from_pandas(train))
valid=Dataset(pa.Table.from_pandas(valid))

In [ ]:
type(train)

datasets.arrow_dataset.Dataset

In [10]:
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
from transformers import TrainingArguments,Trainer
train_args=TrainingArguments(output_dir='/content/gdrive/MyDrive/results',evaluation_strategy='epoch')
trainer=Trainer(
    model=model,
    args=train_args,
    train_dataset=train,
    eval_dataset=valid,
    tokenizer=tokenizer


)

In [12]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.000133
2,No log,0.000084
3,No log,0.000073


TrainOutput(global_step=300, training_loss=0.007783493200937907, metrics={'train_runtime': 241.5297, 'train_samples_per_second': 9.937, 'train_steps_per_second': 1.242, 'total_flos': 631466532864000.0, 'train_loss': 0.007783493200937907, 'epoch': 3.0})

In [13]:
trainer.evaluate()

{'eval_loss': 7.274124800460413e-05,
 'eval_runtime': 6.1301,
 'eval_samples_per_second': 32.626,
 'eval_steps_per_second': 4.078,
 'epoch': 3.0}

In [14]:
model.save_pretrained('./model/')

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
from transformers import AutoModelForSequenceClassification
new_model=AutoModelForSequenceClassification.from_pretrained('./model/').to(device)

In [24]:
from transformers import AutoTokenizer

new_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


In [25]:
def get_prediction(text):
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = new_model(**encoding)

    logits = outputs.logits
    #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    sigmoid = torch.nn.Sigmoid()
    print(sigmoid)
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1)
    
    if label == 1:
        return {
            'sentiment': 'Positive',
            'probability': probs[1]
        }
    else:
        return {
            'sentiment': 'Negative',
            'probability': probs[0]
        }

In [26]:
get_prediction('I am not happy to see you.')

Sigmoid()


{'sentiment': 'Negative', 'probability': 0.9724839}